In [1]:
import cx_Oracle
import os

def login_ck(mlogin,mid,mpw='') :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기    
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    if mlogin == 'diamond' :
        cursor.execute(f"""SELECT mb_id, mb_signup_type, mb_type FROM tbl_member 
                        where mb_signup_type='diamond' and mb_id='{mid}'and mb_pw='{mpw}'""")     
    else :
        cursor.execute(f"SELECT mb_id, mb_signup_type, mb_type FROM tbl_member where mb_signup_type='{mlogin}' and mb_id='{mid}'")
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    
    if len(data):
        data = data[0]
        session['user_id'] =  data[0]
        session['login_by'] = data[1]
        session['manager'] = data[2]
        return redirect(url_for('login_info', info='001777'))
    else :
        return redirect(url_for('login_info', info='001666'))

def select_all(mlogin,mid) :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    cursor.execute(f"""SELECT mb_name, mb_id, mb_phone, mb_email, mb_signup_type, mb_type FROM tbl_member 
                    where mb_signup_type='{mlogin}' and mb_id='{mid}'""")
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    if data:
        data = data[0]
        return data
    else :
        return render_template('/404')
def id_ck(ID) :
    result = False
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    cursor.execute(f"""SELECT count(*) FROM tbl_member 
                    where mb_id='{mid}'""")
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    if data:
        data = data[0]
        print(data)
        if data == 0 :
            result = True
    
    return result

  
def my_cage(mb_id) :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기
    conn = cx_Oracle.connect('diamond','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    cursor.execute(f"""
            SELECT T1.CAGE_SEQ, T2.CAGE_NAME, T1.PET_TYPE, LISTAGG(T1.PET_NAME,',') WITHIN GROUP (ORDER BY T1.PET_NAME) AS PET
            FROM TBL_PET T1, TBL_CAGE T2
            WHERE T1.CAGE_SEQ = T2.CAGE_SEQ and T2.MB_ID = 'dodo'
            GROUP BY T1.PET_TYPE, T1.CAGE_SEQ, T2.CAGE_NAME
            ORDER BY T1.CAGE_SEQ
        """)
    data = cursor.fetchall()
    cursor.close()
    conn.close()
    if data :
        cage_index = set([a[0:2] for a in data])
        cage_list = []
        for cage in cage_index :
            cage_info = {'cage_seq':cage[0], 'cage_name':cage[1], 'pet': []}
            cage_info['pet'] += [{'pet_type':i[2], 'pet_name': i[3]} for i in data if i[0:2] == cage]
            cage_list.append(cage_info)

        return cage_list
    else :
        return None

In [ ]:
from flask import Flask, session, render_template, redirect, request, url_for
from markupsafe import escape
from datetime import timedelta
import random


app = Flask(__name__)
app.secret_key = f"I'm_on_TOP!{random.randint(0, 99999):0>5}"
app.config["PERMANENT_SESSION_LIFETIME"] = timedelta(minutes=60) # 로그인 지속 시간

token = ''
name = ''
info = ''

######### 메인 화면 #########
@app.route('/') # home 
def index() :
    return render_template('home.html', session=session)

@app.route('/<info>')  # home + 이전 페이지 결과
def index_info(info) :
    if len(session) == 0 and info == '001777':
        # session 없는데 로그인 성공 떴을 때
        info = '001000'
    return render_template('home.html', session=session)


######### 회원 가입 #########
@app.route('/signup') # 회원 가입
def signup() :
    if len(session) :
        return redirect(url_for('signup_info', info='705001'))
    return render_template('signup.html', session=session)

@app.route('/signup/confirm') # 회원 가입 + 정보 확인
def confirm_id() :
    return render_template('confirm_id.html')

@app.route('/signup/confirm/<info>') # 회원 가입 + 정보 확인 + 결과
def confirm_info(info) :
    return render_template('confirm_id.html')

@app.route('/signup/confirm/result', methods=['GET']) # 회원 가입 + 정보 확인 + 입력값 체크
def signup_result() :
    if request.method == 'GET' :
        mb_id = request.args['mb_id']
        return id_ck(mb_id)


######### 로그인 #########
@app.route('/login') # 로그인
def login() :
    if len(session) :
        return redirect(url_for('login_info', info='705001'))
    return render_template('login.html')

@app.route('/login/<info>') #로그인 + 이전 페이지 결과
def login_info(info) :
    return render_template('login_result.html')
    

@app.route('/login/result', methods=['POST']) # 로그인 + 입력값 체크
def login_result() :
    if request.method == 'POST' :
        mlogin = request.form['mlogin']
        mid = request.form['mid']
        mpw = ''
        if mlogin == 'naver' : 
            token = request.form['token']
        elif mlogin == 'diamond' :
            mpw = request.form['mpw']
        return login_ck(mlogin,mid,mpw)

    
######### 로그아웃 #########
@app.route('/logout') # 로그아웃
def logout() :
    if len(session) :
        session.pop('user_id', None)
        session.pop('login_by', None)
        session.pop('manager', None)
        if len(session) == 0 :
            info = '000777'
        else :
            info = '000666'
    else :
        info = "000000"
    print(session)
    print(f'로그아웃 : {len(session)}')
    return redirect(url_for('index_info', info=info))



########## 마이페이지 ##########
@app.route('/mypage/cage') #사육장 목록
def mycage() :
    if len(session) :
        cage_list = my_cage(session['user_id'])
        return render_template('mypage_cage.html', session=session, cage_list=cage_list)
    else :
        return redirect(url_for_for('index_info', info='705000')) 
    
    

######### 기타 페이지 #########
@app.route('/diamond/team') # 팀소개
def diamond() :
    return render_template('diamond_team.html', session=session)

@app.route('/diamond/shop') # 쇼핑몰
def shop() :
    return render_template('diamond_shop.html', session=session)

@app.route('/diamond/contact') # 오시는 길
def contact() :
    return render_template('diamond_contact.html', session=session)

@app.route('/community/free') # 자유게시판
def community() :
    return render_template('community_free.html', session=session)


######### 에러 처리 #########
@app.errorhandler(404)
def page_not_found(e):
    return render_template("404.html"), 404    
    
@app.errorhandler(500)
def page_not_found(e):
    return render_template("500.html"), 500

if __name__ == '__main__' :
    app.run(host = '0.0.0.0', port=5020)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.70.64:5020/ (Press CTRL+C to quit)
192.168.70.64 - - [26/Aug/2022 21:38:11] "GET / HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:38:12] "GET /static/img/lizard3.jpg HTTP/1.1" 304 -
192.168.70.64 - - [26/Aug/2022 21:38:12] "GET /static/img/lizard1.jpg HTTP/1.1" 304 -
192.168.70.64 - - [26/Aug/2022 21:38:12] "GET /static/img/lizard2.jpg HTTP/1.1" 304 -
192.168.70.64 - - [26/Aug/2022 21:38:12] "GET /static/img/lizard4.jpg HTTP/1.1" 304 -
192.168.70.64 - - [26/Aug/2022 21:38:12] "GET /static/img/lizard5.jpg HTTP/1.1" 304 -
192.168.70.64 - - [26/Aug/2022 21:38:12] "GET /static/img/lizard6.png HTTP/1.1" 304 -
192.168.70.64 - - [26/Aug/2022 21:38:12] "GET /favicon.ico HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:38:23] "GET /diamond/contact HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:38:24] "GET /favicon.ico HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:38:27] "GET /static/bootstrap.bundle.min.js.map HTTP/1.1" 200 -


192.168.70.64 - - [26/Aug/2022 21:58:26] "GET /static/img/map.png HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:58:26] "GET /static/img/tel.png HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:58:26] "GET /static/img/email.png HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:58:26] "GET /static/bootstrap.bundle.min.js.map HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:58:27] "GET /favicon.ico HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:59:36] "GET /signup HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:59:36] "GET /static/bootstrap.min.css HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:59:36] "GET /static/bootstrap.css HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:59:36] "GET /static/common.css HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:59:36] "GET /static/common.js HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:59:36] "GET /static/footer.css HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 21:59:36] "GET /static/bootstrap.bundle.min.js HTTP/1.1" 200 -
1

192.168.70.64 - - [26/Aug/2022 22:01:29] "GET /static/img/email.png HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 22:01:30] "GET /static/bootstrap.bundle.min.js.map HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 22:01:30] "GET /favicon.ico HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 22:02:50] "GET /signup HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 22:02:50] "GET /static/bootstrap.min.css HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 22:02:50] "GET /static/bootstrap.css HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 22:02:50] "GET /static/footer.css HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 22:02:50] "GET /static/common.css HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 22:02:50] "GET /static/bootstrap.bundle.min.js HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 22:02:50] "GET /static/common.js HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 22:02:50] "GET /static/prism.js HTTP/1.1" 200 -
192.168.70.64 - - [26/Aug/2022 22:02:50] "GET /static/sweetalert.min.js HTTP/1.1" 200 